fr

In [1]:
import json
from decoding import istdeco_decode
from imaging_utils import ISSDataContainer
import numpy as np
import pandas as pd
from decoding import estimate_fdr

# Create the codebook
num_rounds = 5
num_channels = 5


data = json.load(open(r'..\..\ISTDECO\downloads\liver_2d\metadata.json','r'))['target_data']
codebook = np.zeros((len(data), num_rounds, num_channels))
gene_names = []
for i, item in enumerate(data):
    channels = []
    gene_names.append(item['names'])
    for r in range(num_rounds):
        for c in range(num_channels):
            codebook[i,r,c] = item[f'r{r}_c{c}']

dataset = ISSDataContainer()
dataset.add_images_from_filepattern('..\\..\\ISTDECO\\ASHLAR\\ASHLAR_R{round}_C{channel}.tif')
dataset.load()


Added ..\..\ISTDECO\ASHLAR\ASHLAR_R0_C0.tif. Stage: 0, Round: 0, Channel: 0
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R0_C1.tif. Stage: 0, Round: 0, Channel: 1
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R0_C2.tif. Stage: 0, Round: 0, Channel: 2
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R0_C3.tif. Stage: 0, Round: 0, Channel: 3
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R0_C4.tif. Stage: 0, Round: 0, Channel: 4
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R1_C0.tif. Stage: 0, Round: 1, Channel: 0
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R1_C1.tif. Stage: 0, Round: 1, Channel: 1
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R1_C2.tif. Stage: 0, Round: 1, Channel: 2
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R1_C3.tif. Stage: 0, Round: 1, Channel: 3
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R1_C4.tif. Stage: 0, Round: 1, Channel: 4
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R2_C0.tif. Stage: 0, Round: 2, Channel: 0
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R2_C1.tif. Stage: 0, Round: 2, Channel: 1
Added ..\..\ISTDECO\ASHLAR\ASHLAR_R2_C2.tif. Stage: 0, Round: 2, Channel: 2
Added ..\..\

In [16]:
dfs = []
for k, (slice, loc) in enumerate(dataset.iterate_tiles(512, 512)):
    if k == 30:
        data = slice.squeeze()
        output = istdeco_decode(data, codebook, (1.50, 1.50), lowpass_sigma=(7.0, 7.0), min_integrated_intensity=1000, min_correct_spots=3.5)
        output['X'] = output['X'] + loc[1]
        output['Y'] = output['Y'] + loc[0]
        output['Gene'] = [gene_names[id] for id in output['Target id']]
        dfs.append(pd.DataFrame(output))
        print(k)
dfs = pd.concat(dfs)
labels = dfs['Gene'].to_list()
neg = [g for g in gene_names if 'Blank' in g]
pos = [g for g in gene_names if 'Blank' not in g]
fdr = estimate_fdr(labels, neg, pos)
print(f'FDR: {fdr}, N: {len(labels)}')
#dfs.to_csv('..\\..\\ISTDECO\\ASHLAR\\result.csv', index=False)


30
FDR: 0.030677290836042287, N: 502


In [7]:
from channel_utils import ChannelAnalyzer

dfs = []
for k, (slice, loc) in enumerate(dataset.iterate_tiles(512, 512)):
    if k == 30:
        data = slice.squeeze()
        cross = []
        for r in range(5):
            ch = ChannelAnalyzer(data[r].copy())
            cross.append(np.diag(np.diag(ch.crosstalk_matrix)))
        cross = np.array(cross)
        cross[:,4,:] = 0
        cross[:,:,4] = 0

        output = istdeco_decode(data, codebook, (2.50, 2.50), lowpass_sigma=(7.0, 7.0), min_integrated_intensity=1000, min_correct_spots=3.7, radius=2)
        output['X'] = output['X'] + loc[1]
        output['Y'] = output['Y'] + loc[0]
        output['Gene'] = [gene_names[id] for id in output['Target id']]
        dfs.append(pd.DataFrame(output))
        print(k)
dfs = pd.concat(dfs)
labels = dfs['Gene'].to_list()
neg = [g for g in gene_names if 'Blank' in g]
pos = [g for g in gene_names if 'Blank' not in g]
fdr = estimate_fdr(labels, neg, pos)
print(f'FDR: {fdr}, N: {len(labels)}')
#dfs.to_csv('..\\..\\ISTDECO\\ASHLAR\\result.csv', index=False)


30
FDR: 0.14392523364149706, N: 428


In [6]:
fdr = estimate_fdr(labels, neg, pos)
print(f'FDR: {fdr}, N: {len(labels)}')


FDR: 0.24131054130710386, N: 702


In [14]:
cross[0,:,:]

array([[0.70206635, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.89818449, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.73230722, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])